In [1]:
import pandas as pd
import polars as pl
import fastf1 as ff1
from tqdm import tqdm

In [2]:
#race_data2 = pd.concat([first_batch,second_batch,third_batch,fourth_batch])
race_data = pd.read_parquet("./static/data/tester.parquet")

In [3]:
grouped_race_data = race_data.groupby(["round_number","year","driver_number"]).count().sort_values(by=["year","round_number","pos_index"], ascending=[True,True,False])

In [4]:
grouped_race_data

LapNumber      x      y  pos_index  \
round_number year driver_number                                       
1            2020 6                    294  25698  25698      25698   
                  77                   297  25691  25691      25691   
                  5                    291  25032  25032      25032   
                  99                   302  25016  25016      25016   
                  31                   290  24999  24999      24999   
...                                    ...    ...    ...        ...   
8            2024 16                   298  33257  33257      33257   
                  11                     2   9445   9445       9445   
                  20                     2   9445   9445       9445   
                  27                     2   9445   9445       9445   
                  31                     2   9445   9445       9445   

                                 artifical_pos  
round_number year driver_number                 
1            2020 6                      25698  
                  77                     25691  
                  5                      25032  
                  99                     25016  
                  31                     24999  
...                                        ...  
8            2024 16                     33257  
                  11                      9445  
                  20                      9445  
                  27                      9445  
                  31                      9445  

[1794 rows x 5 columns]

In [5]:
max_number_data_points_per_race = grouped_race_data.groupby(["year","round_number"])["x"].max().reset_index()

In [6]:
max_number_data_points_per_race = max_number_data_points_per_race.rename(columns={"x":"max_datapoints"})

In [7]:
max_number_data_points_per_race

,year,round_number,max_datapoints
0,2020,1,25698
1,2020,2,22949
2,2020,3,26676
3,2020,4,24330
4,2020,5,22140
...,...,...,...
85,2024,4,26576
86,2024,5,23676
87,2024,6,21133
88,2024,7,19929


Write function to append the last known datapoint until the DataFrame for the driver is as long as the longest DataFrame

In [8]:
pl_race_data = pl.from_pandas(race_data).lazy()

In [9]:
new_race_data_list = []
for gp,year, driver_number in tqdm(grouped_race_data.index):
    max_datapoints = max_number_data_points_per_race.loc[(max_number_data_points_per_race["year"] == year) & (max_number_data_points_per_race["round_number"] == gp),"max_datapoints"].item()
    filtered_race_data = pl_race_data.filter((pl.col("year")==year)&(pl.col("round_number")==gp)&(pl.col("driver_number")==driver_number)).collect().to_pandas()
    diff_max_points = max_datapoints-len(filtered_race_data)
    filtered_race_data["artifical_pos"] = False

    if diff_max_points != 0:
        df_to_add = pd.DataFrame([filtered_race_data.iloc[-1]]*diff_max_points)
        df_to_add["artifical_pos"] = True
        new_race_data_list.append(pd.concat(
            [filtered_race_data,df_to_add]
        ))
    else: 
        new_race_data_list.append(filtered_race_data)


100%|██████████| 1794/1794 [01:15<00:00, 23.68it/s]


In [10]:
modified_race_data = pd.concat(new_race_data_list)

In [11]:
modified_race_data.groupby(["round_number","year","driver_number"]).count().sort_values(by=["year","round_number","x"], ascending=[True,True,False])


LapNumber      x      y  pos_index  \
round_number year driver_number                                       
1            2020 3                     53  25698  25698      25698   
                  4                    300  25698  25698      25698   
                  5                    291  25698  25698      25698   
                  6                    294  25698  25698      25698   
                  7                    221  25698  25698      25698   
...                                    ...    ...    ...        ...   
8            2024 44                   299  33486  33486      33486   
                  55                   303  33486  33486      33486   
                  63                   323  33486  33486      33486   
                  77                   297  33486  33486      33486   
                  81                   302  33486  33486      33486   

                                 artifical_pos  
round_number year driver_number                 
1            2020 3                      25698  
                  4                      25698  
                  5                      25698  
                  6                      25698  
                  7                      25698  
...                                        ...  
8            2024 44                     33486  
                  55                     33486  
                  63                     33486  
                  77                     33486  
                  81                     33486  

[1794 rows x 5 columns]

In [12]:
modified_race_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44959521 entries, 0 to 9444
Data columns (total 8 columns):
 #   Column         Dtype  
---  ------         -----  
 0   round_number   int8   
 1   year           int16  
 2   driver_number  int8   
 3   LapNumber      float64
 4   x              float64
 5   y              float64
 6   pos_index      uint16 
 7   artifical_pos  bool   
dtypes: bool(1), float64(3), int16(1), int8(2), uint16(1)
memory usage: 1.6 GB


In [13]:
modified_race_data.to_parquet("./static/data/tester.parquet", index=False)

In [14]:
modified_race_data.reset_index(drop=True).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44959521 entries, 0 to 44959520
Data columns (total 8 columns):
 #   Column         Dtype  
---  ------         -----  
 0   round_number   int8   
 1   year           int16  
 2   driver_number  int8   
 3   LapNumber      float64
 4   x              float64
 5   y              float64
 6   pos_index      uint16 
 7   artifical_pos  bool   
dtypes: bool(1), float64(3), int16(1), int8(2), uint16(1)
memory usage: 1.3 GB


In [17]:
modified_race_data.ffill().to_parquet("race_data.parquet")